In [88]:
import gc
import os
import time
import logging
import datetime
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
from scipy import stats
from scipy.signal import hann
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from scipy.signal import hilbert
from scipy.signal import convolve
from sklearn.svm import NuSVR, SVR
from xgboost.sklearn import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,StratifiedKFold, RepeatedKFold
from sklearn.ensemble import RandomForestRegressor
warnings.filterwarnings("ignore")

In [140]:
fulldf=pd.read_csv('D:/NINI data/data/full_DF.csv')

In [141]:
fulldf.head()

,label,MEAN_full,SD_full,q0_full,q0.01_full,q0.05_full,q0.1_full,q0.25_full,q0.5_full,q0.75_full,...,SD_roll_sd_1500,q0_roll_sd_1500,q0.05_roll_sd_1500,q0.25_roll_sd_1500,q0.75_roll_sd_1500,q0.95_roll_sd_1500,q1_roll_sd_1500,ar1_roll_sd_1500,kur_roll_sd_1500,skew_roll_sd_1500
0,1.430797,4.884113,5.101106,-98,-8.0,-2.0,0.0,3,5.0,7.0,...,2.656865,2.617320,2.736570,2.995398,4.788156,7.817831,25.824677,0.999997,30.344736,4.762917
1,1.411696,4.746800,6.173383,-154,-9.0,-2.0,0.0,2,5.0,7.0,...,4.043365,2.617320,2.727464,2.987012,4.789842,8.687332,32.139798,0.999995,28.472282,5.088456
2,1.391499,4.725767,6.588824,-154,-11.0,-2.0,0.0,2,5.0,7.0,...,4.337875,2.592122,2.702003,2.966430,5.295588,10.766482,32.139798,0.999995,20.393856,4.228071
3,1.372297,4.753000,5.715036,-115,-11.0,-2.0,0.0,2,5.0,7.0,...,3.103912,2.592122,2.694748,2.993000,5.308373,11.257134,18.720802,0.999994,5.218347,2.257274
4,1.353196,4.906393,6.967397,-106,-15.0,-3.0,0.0,2,5.0,7.0,...,4.164771,2.619431,2.798193,3.092818,6.069799,14.188273,29.323379,0.999996,8.075628,2.556367


In [138]:
alltest=pd.read_csv('D:/NINI data/data/alltest.csv')

In [139]:
alltest.head()

,MEAN_full,SD_full,q0_full,q0.01_full,q0.05_full,q0.1_full,q0.25_full,q0.5_full,q0.75_full,q0.9_full,...,SD_roll_sd_1500,q0_roll_sd_1500,q0.05_roll_sd_1500,q0.25_roll_sd_1500,q0.75_roll_sd_1500,q0.95_roll_sd_1500,q1_roll_sd_1500,ar1_roll_sd_1500,kur_roll_sd_1500,skew_roll_sd_1500
0,4.491780,4.893690,-75,-8.0,-2,0.0,2.0,4,7,9,...,2.292318,2.570801,2.792225,3.038639,4.433796,8.156856,20.448716,0.999995,18.830727,3.678800
1,4.171153,5.922839,-140,-12.0,-2,-1.0,2.0,4,7,9,...,3.599575,2.588071,2.758785,2.987653,4.551518,12.945506,24.631268,0.999995,11.786130,3.322717
2,4.610260,6.946990,-193,-11.0,-2,0.0,2.0,5,7,9,...,5.023551,2.596135,2.715449,2.943750,4.432930,13.707627,43.596921,0.999995,30.025207,5.072505
3,4.531473,4.114147,-93,-5.0,-1,1.0,3.0,5,7,8,...,2.051264,2.489678,2.633196,2.715904,3.245881,7.228895,18.862115,0.999994,24.018188,4.336000
4,4.128340,5.797164,-147,-10.0,-2,0.0,2.0,4,6,9,...,3.683712,2.561236,2.654986,2.857051,4.340566,10.267064,28.828019,0.999996,18.076352,3.942079


In [142]:
train_X=fulldf.iloc[:,1:]
train_y=fulldf.iloc[:,0]

In [143]:
train_y.head()

0    1.430797
1    1.411696
2    1.391499
3    1.372297
4    1.353196
Name: label, dtype: float64

In [144]:
scaler = StandardScaler()
scaler.fit(train_X)
scaled_train_X = pd.DataFrame(scaler.transform(train_X), columns=train_X.columns)

In [145]:
scaled_test_X = pd.DataFrame(scaler.transform(alltest), columns=alltest.columns)

In [146]:
scaled_train_X.shape

(8252, 82)

In [124]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=42)
train_columns = scaled_train_X.columns.values

In [57]:
params = {'num_leaves': 51,
         'min_data_in_leaf': 10, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.001,
         "boosting": "gbdt",
         "feature_fraction": 0.91,
         "bagging_freq": 1,
         "bagging_fraction": 0.91,
         "bagging_seed": 42,
         "metric": 'mae',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": -1,
         "random_state": 42}

In [58]:
oof = np.zeros(len(scaled_train_X))
predictions = np.zeros(len(scaled_test_X))
feature_importance_df = pd.DataFrame()
#run model
for fold_, (trn_idx, val_idx) in enumerate(folds.split(scaled_train_X,train_y.values)):
    strLog = "fold {}".format(fold_)
    print(strLog)
    
    X_tr, X_val = scaled_train_X.iloc[trn_idx], scaled_train_X.iloc[val_idx]
    y_tr, y_val = train_y.iloc[trn_idx], train_y.iloc[val_idx]

    model = lgb.LGBMRegressor(**params, n_estimators = 30000, n_jobs = -1)
    model.fit(X_tr, 
              y_tr, 
              eval_set=[(X_tr, y_tr), (X_val, y_val)], 
              eval_metric='mae',
              verbose=1000, 
              early_stopping_rounds=500)
    oof[val_idx] = model.predict(X_val, num_iteration=model.best_iteration_)
    #feature importance
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = train_columns
    fold_importance_df["importance"] = model.feature_importances_[:len(train_columns)]
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    #predictions
    predictions += model.predict(scaled_test_X, num_iteration=model.best_iteration_) / folds.n_splits


fold 0
Training until validation scores don't improve for 500 rounds.
[1000]	training's l1: 2.03464	valid_1's l1: 2.22377
[2000]	training's l1: 1.75678	valid_1's l1: 2.08772
[3000]	training's l1: 1.60865	valid_1's l1: 2.055
[4000]	training's l1: 1.49587	valid_1's l1: 2.04123
[5000]	training's l1: 1.39941	valid_1's l1: 2.02944
[6000]	training's l1: 1.31426	valid_1's l1: 2.02021
[7000]	training's l1: 1.2382	valid_1's l1: 2.01257
[8000]	training's l1: 1.16865	valid_1's l1: 2.0057
[9000]	training's l1: 1.10387	valid_1's l1: 2.00009
[10000]	training's l1: 1.04352	valid_1's l1: 1.99491
[11000]	training's l1: 0.987648	valid_1's l1: 1.99064
[12000]	training's l1: 0.935465	valid_1's l1: 1.98674
[13000]	training's l1: 0.88655	valid_1's l1: 1.98345
[14000]	training's l1: 0.840627	valid_1's l1: 1.98052
[15000]	training's l1: 0.797324	valid_1's l1: 1.97789
[16000]	training's l1: 0.756722	valid_1's l1: 1.97542
[17000]	training's l1: 0.718215	valid_1's l1: 1.97292
[18000]	training's l1: 0.682326	vali

In [59]:
submission = pd.read_csv('D:/NINI data/data/sample_submission.csv', index_col='seg_id')

In [60]:
submission.time_to_failure = predictions
submission.to_csv('D://NINI data/submission3_1.csv',index=True)

In [ ]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_X,train_y.values)):
    strLog = "fold {}".format(fold_)
    print(strLog)
    X_tr, X_val = train_X.iloc[trn_idx], train_X.iloc[val_idx]
    y_tr, y_val = train_y.iloc[trn_idx], train_y.iloc[val_idx]
    model = RandomForestRegressor(n_estimators=1000,criterion='mae')
    model.fit(X_tr, y_tr) 
            
    print('mae',mean_absolute_error(train_y.iloc[val_idx],model.predict(train_X.iloc[val_idx])))
    

### xgboost

In [147]:
xgb1 = XGBRegressor(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 nthread=4,
 scale_pos_weight=1,
 silent=False,
 seed=27)
xgb1.fit(scaled_train_X, train_y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
       nthread=4, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=27, silent=False,
       subsample=0.8)

In [148]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBRegressor( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'reg:linear', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='neg_mean_absolute_error',n_jobs=4,iid=False, cv=5)
gsearch1.fit(scaled_train_X,train_y)
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

({'mean_fit_time': array([2.60504608, 2.19441547, 2.16751633, 3.29120989, 3.38847861,
         3.21593137, 4.52636485, 4.53068171, 4.61298246, 5.89338713,
         5.91108537, 4.93666706]),
  'std_fit_time': array([0.27313259, 0.12521519, 0.07757047, 0.36674265, 0.06193949,
         0.25931483, 0.36915918, 0.27631319, 0.36464637, 0.33900171,
         0.35216067, 0.86468336]),
  'mean_score_time': array([0.01098237, 0.0090786 , 0.01017656, 0.00997615, 0.01147041,
         0.00997567, 0.01246767, 0.01206894, 0.01198506, 0.01336784,
         0.01336818, 0.01276588]),
  'std_score_time': array([0.00328681, 0.00102006, 0.00172174, 0.00090038, 0.00227371,
         0.00063084, 0.00147921, 0.00149365, 0.00070761, 0.00049297,
         0.00119861, 0.00132302]),
  'param_max_depth': masked_array(data=[3, 3, 3, 5, 5, 5, 7, 7, 7, 9, 9, 9],
               mask=[False, False, False, False, False, False, False, False,
                     False, False, False, False],
         fill_value='?',
         

In [149]:
y_pred_xg = gsearch1.predict(scaled_test_X)

In [156]:
y_pred_xg[1:5]

array([5.8376064, 5.22111  , 8.1089735, 6.775595 ], dtype=float32)

In [129]:
submission['time_to_failure'] = y_pred
submission.to_csv('D://NINI data//submission_xgboost2.csv')

In [98]:
scaled_train_X.head()

,mean_full,sd_full,X_full,X_full.1,q0_full,q0.01_full,q0.05_full,q0.1_full,q0.25_full,q0.5_full,...,X_roll_sd_1500.1,q0_roll_sd_1500,q0.05_roll_sd_1500,q0.25_roll_sd_1500,q0.75_roll_sd_1500,q0.95_roll_sd_1500,q1_roll_sd_1500,ar1_roll_sd_1500,kur_roll_sd_1500,skew_roll_sd_1500
0,1.417288,-0.174309,-0.220174,0.196545,0.196545,0.239325,0.090408,0.182526,1.597236,0.948201,...,0.661393,0.661393,0.096891,0.096992,0.018557,-0.538557,-0.112766,1.154002,0.720015,0.609325
1,0.880347,-0.049123,0.058938,-0.011966,-0.011966,0.166802,0.090408,0.182526,-0.398281,0.948201,...,0.661393,0.661393,0.015769,0.067238,0.019562,-0.401818,-0.031079,0.016677,0.588726,0.852693
2,0.798100,-0.000622,0.058938,-0.011966,-0.011966,0.021756,0.090408,0.182526,-0.398281,0.948201,...,0.286952,0.286952,-0.211063,-0.005789,0.321140,-0.074850,-0.031079,-0.332303,0.022298,0.209482
3,0.904591,-0.102634,-0.194800,0.133247,0.133247,0.021756,0.090408,0.182526,-0.398281,0.948201,...,0.286952,0.286952,-0.275703,0.088486,0.328764,0.002311,-0.204657,-0.756343,-1.041749,-1.263855
4,1.504410,0.043576,-0.089680,0.166758,0.166758,-0.268336,-0.325657,0.182526,-0.398281,0.948201,...,0.692766,0.692766,0.645891,0.442644,0.782805,0.463264,-0.067510,0.138341,-0.841408,-1.040257


(8252,)

In [151]:
rfc_model = RandomForestRegressor(random_state=27).fit(scaled_train_X,train_y)
y_pred_rf = rfc_model.predict(scaled_test_X)


In [153]:
mean_absolute_error(train_y,rfc_model.predict(scaled_train_X))

0.8171374198404829

In [154]:
y_pred_comb=(y_pred_rf+y_pred_xg)/2

In [157]:
y_pred_comb[1:5]

array([5.39269219, 5.36373917, 8.18500087, 6.98568148])

In [158]:
submission['time_to_failure'] = y_pred_comb
submission.to_csv('D://NINI data//submission_xgb_rf.csv')